In [24]:
import time
from utils.vector_db import vectorDataBase
from tqdm import tqdm
from langchain_google_genai import GoogleGenerativeAI
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_community.document_loaders import PDFMinerLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

# **LLM's**

In [2]:
llm = GoogleGenerativeAI(model="gemini-2.5-flash", verbose=True)

In [ ]:
print(llm.invoke("Can you teach me bankai??"))

# **Vector Stores**

In [4]:
embeddings = GoogleGenerativeAIEmbeddings(
    model="models/gemini-embedding-001",
    task_type="retrieval_document"
)

db_creator = vectorDataBase()
pdf_store = db_creator.initalize_empty_db(
    emdeddingFunction=embeddings
)

Previous vector db is over written by this function


# **Loading from websites**

In [26]:
loader = PDFMinerLoader(
    r"E:\jjk_pdf.pdf"
)
loaded_docs = loader.load()

In [27]:
splitter = RecursiveCharacterTextSplitter(
    chunk_size=200,
    chunk_overlap=50,
    length_function=len
)

docs = splitter.split_documents(loaded_docs)
print(len(docs))

648


In [32]:
with tqdm(total=len(docs), desc="Adding docs", unit="docs") as pbar:
    for i in range(0, len(docs), 10):
        batches = docs[i:i+20]
        length = len(pdf_store.add_documents(batches))
        time.sleep(30)
        pbar.update(length)

Adding docs:   0%|          | 0/648 [00:00<?, ?docs/s]


GoogleGenerativeAIError: Error embedding content: 429 Resource has been exhausted (e.g. check quota).

In [ ]:
retiever = pdf_store.as_retriever(k=5)

In [ ]:
retiever.invoke("is sukuna really villan?")

[Document(id='aa33f558-2349-46ca-8fad-98f211661763', metadata={'source': 'https://jujutsu-kaisen.fandom.com/wiki/Jujutsu_Kaisen', 'title': 'Jujutsu Kaisen | Jujutsu Kaisen Wiki | Fandom', 'description': 'Jujutsu Kaisen (呪 (じゅ) 術 (じゅつ) 廻 (かい) 戦 (せん) , Jujutsu Kaisen?)\xa0is a manga written and illustrated by\xa0Gege Akutami\xa0and is published in\xa0Weekly Shonen Jump. The first chapter was published on March 5, 2018, in issue 14 of\xa0Weekly Shonen Jump. The final chapter was published on September 30, 2024...', 'language': 'en'}, page_content='Characters\nMain Article: List of Characters\nYuji ItadoriMegumi FushiguroNobara KugisakiSatoru GojoSukuna\nVolumes\nMain article: Volumes & Chapters'),
 Document(id='5e0943c2-cd45-4ec7-b2c9-ebf978685a28', metadata={'source': 'https://jujutsu-kaisen.fandom.com/wiki/Jujutsu_Kaisen', 'title': 'Jujutsu Kaisen | Jujutsu Kaisen Wiki | Fandom', 'description': 'Jujutsu Kaisen (呪 (じゅ) 術 (じゅつ) 廻 (かい) 戦 (せん) , Jujutsu Kaisen?)\xa0is a manga written and il

In [ ]:
pdf_store.save_local(r"db")